In [1]:
# import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
from textwrap import wrap
import re
import jdatetime

In [2]:
# URLs
file_path = './data/Industry-Indices.xlsx'
tse_main_url = 'http://www.tsetmc.com/Loader.aspx?ParTree=15'
industries_operation_url = 'http://www.tsetmc.com/Loader.aspx?Partree=15131O'

## Fetching data

In [3]:
res = requests.get(url=tse_main_url)
res.text[:300]

'<!doctype html><html><head><title>.:TSETMC:. :: شرکت مدیریت فناوری بورس تهران</title><script>var LongRunnigPagesSite=\'http://cdn2.tsetmc.com\';function ens(ty,sv){var lv=localStorage.getItem("v_"+ty);var t;if (1==1 || lv!=sv){var oX=new XMLHttpRequest();oX.open(\'GET\', \'tsev2/res/loader.aspx?t=\'+ty+\'&'

In [4]:
def get_and_parse_url(url, params=None):
    """
        get a URL, grab the data, and return a BeautifulSoup object with parsed data
    """
    res = requests.get(url=url, params=params)
    soup = BeautifulSoup(markup=res.text, features='html.parser')
    return soup

In [5]:
soup = get_and_parse_url(url=industries_operation_url)
print(soup.prettify()[:500])

<!DOCTYPE doctype html>
<html>
 <head>
  <title>
   .:TSETMC:. :: برترین گروه های صنعت
  </title>
  <script>
   var LongRunnigPagesSite='http://cdn2.tsetmc.com';function ens(ty,sv){var lv=localStorage.getItem("v_"+ty);var t;if (1==1 || lv!=sv){var oX=new XMLHttpRequest();oX.open('GET', 'tsev2/res/loader.aspx?t='+ty+'&_'+sv,false);oX.send(null);t=oX.responseText;if(t[t.length-1]!=';') return;localStorage.setItem("v_"+ty,sv);localStorage.setItem("t_"+ty,t)}else{t=localStorage.getItem("t_"+ty)}if (


In [6]:
msoup = get_and_parse_url(url=tse_main_url)
print(msoup.prettify()[:500])

<!DOCTYPE doctype html>
<html>
 <head>
  <title>
   .:TSETMC:. :: شرکت مدیریت فناوری بورس تهران
  </title>
  <script>
   var LongRunnigPagesSite='http://cdn2.tsetmc.com';function ens(ty,sv){var lv=localStorage.getItem("v_"+ty);var t;if (1==1 || lv!=sv){var oX=new XMLHttpRequest();oX.open('GET', 'tsev2/res/loader.aspx?t='+ty+'&_'+sv,false);oX.send(null);t=oX.responseText;if(t[t.length-1]!=';') return;localStorage.setItem("v_"+ty,sv);localStorage.setItem("t_"+ty,t)}else{t=localStorage.getItem("t_"


## Find the date (Jalali) and convert it to Gregorian

In [7]:
# Market info located in a blue div
blue_div = msoup.find_all(name='div', class_='box1 blue tbl z1_4 h210')
len(blue_div)

2

In [8]:
# first blue div is the one that I need
blue_div[0]

<div class="box1 blue tbl z1_4 h210">
<div class="header">بازار نقدی بورس در یک نگاه</div>
<div class="content">
<table class="table1">
<tbody>
<tr>
<td>وضعیت بازار</td>
<td>بسته <span class="RealServerTime"><!--RealServerTime--></span></td>
</tr>
<tr>
<td>شاخص کل</td>
<td>229,808.83 <div class="pn">2920.73</div></td>
</tr>
<tr>
<td>شاخص كل (هم وزن)</td>
<td>55,976.26 <div class="pn">1534.65</div></td>
</tr>
<tr>
<td>ارزش بازار</td>
<td><div class="ltr" title="8,723,386,601,944,819">8,723,386.602 B</div></td>
</tr>
<tr>
<td>اطلاعات قیمت</td>
<td>98/3/19 16:08:38</td>
</tr>
<tr>
<td>تعداد معاملات</td>
<td>291,939</td>
</tr>
<tr>
<td>ارزش معاملات</td>
<td><div class="ltr" title="14,719,301,430,525">14,719.301 B</div></td>
</tr>
<tr>
<td>حجم معاملات</td>
<td><div class="ltr" title="4,065,458,593">4.065 B</div></td>
</tr>
</tbody>
</table>
</div>
</div>

In [9]:
# Grab the datetime out of the blue div
tds = blue_div[0].find_all('td')
info_datetime = tds[9].string
info_datetime = '13'+ info_datetime
info_datetime

'1398/3/19 16:08:38'

In [10]:
# throw out the time part and clean the date part
info_j_date = info_datetime[:10]
info_j_date = info_j_date.strip().split('/')
info_j_date[1] = ('0' + info_j_date[1]) if len(info_j_date[1]) < 2 else info_j_date[1]
info_j_date[2] = ('0' + info_j_date[2]) if len(info_j_date[2]) < 2 else info_j_date[2]
info_j_date

['1398', '03', '19']

In [11]:
# convert Jalali date to Gregorian date
info_c_date = jdatetime.date(year=int(info_j_date[0]), 
                             month=int(info_j_date[1]), 
                             day=int(info_j_date[2])).togregorian()
info_c_date

datetime.date(2019, 6, 9)

In [12]:
# convert both Jalali and Gregorian date to string
info_c_date = info_c_date.strftime('%Y-%m-%d')
info_j_date = '{}-{}-{}'.format(info_j_date[0], info_j_date[1], info_j_date[2])
print(info_c_date)
print(info_j_date)

2019-06-09
1398-03-19


## Searching for required data and preparing them

In [13]:
rows = soup.tbody.find_all(name='tr')
len(rows)

41

In [14]:
print(rows[:2])

[<tr><td>شاخص صنعت</td>
<td><div class="ltr" title="7,791,563,766,294,819">7,791,563.766 B</div></td>
<td>211,661</td>
<td><div class="ltr" title="2,099,442,477">2.099 B</div></td>
<td><div class="ltr" title="9,250,453,444,196">9,250.453 B</div></td>
</tr>, <tr><td>34-خودرو</td>
<td><div class="ltr" title="283,194,663,505,523">283,194.664 B</div></td>
<td>31,946</td>
<td><div class="ltr" title="641,540,073">641.540 M</div></td>
<td><div class="ltr" title="1,439,950,428,302">1,439.950 B</div></td>
</tr>]


In [15]:
rows = [r.find_all('td') for r in rows]

In [16]:
# col1: Group
# col2: Market-Value
# col3: Transactions-Number
# col4: Transactions-Volume
# col5: Transactions-Value
rows[1]

[<td>34-خودرو</td>,
 <td><div class="ltr" title="283,194,663,505,523">283,194.664 B</div></td>,
 <td>31,946</td>,
 <td><div class="ltr" title="641,540,073">641.540 M</div></td>,
 <td><div class="ltr" title="1,439,950,428,302">1,439.950 B</div></td>]

In [17]:
# get cells string and save them in a list
values = []
for row in rows:
    for col in row:
        values.append(col.string)
        
values[:10]

['شاخص صنعت',
 '7,791,563.766 B',
 '211,661',
 '2.099 B',
 '9,250.453 B',
 '34-خودرو',
 '283,194.664 B',
 '31,946',
 '641.540 M',
 '1,439.950 B']

In [18]:
# split the values into sized 5 chunks to represent each row in a list item
values = [values[i:i+5] for i in range(0, len(values), 5)]
values[:2]

[['شاخص صنعت', '7,791,563.766 B', '211,661', '2.099 B', '9,250.453 B'],
 ['34-خودرو', '283,194.664 B', '31,946', '641.540 M', '1,439.950 B']]

In [19]:
# remove the ',', 'B' and 'M' from the recieved string and convert it to a float number
def purify_number(number):
    number = str(number)
    number = number.split(',')
    number = ''.join(number)
    number = number.strip()

    if 'B' in number:
        number = number.strip('B')
        number = float(number) * 1000
    elif 'M' in number:
        number = number.strip('M')
        #number = float(number) * 1000000
    else:
        number = float(number)

    return number

In [20]:
a = purify_number('   141.383 B    ')
a

141383.0

In [21]:
# cleaning all the numbers
for i in range(len(values)):
    for j in range(1, len(values[0])):
        values[i][j] = purify_number(values[i][j])

In [22]:
# now data is clean and ready to save
values[:3]

[['شاخص صنعت', 7791563766.0, 211661.0, 2099.0, 9250453.0],
 ['34-خودرو', 283194664.0, 31946.0, '641.540 ', 1439950.0],
 ['44-شيميايي', 1897530485.0, 23884.0, '208.416 ', 1095539.0]]

## Saving the clean data into excel file

In [23]:
# read the original excel file
df_main = pd.read_excel(file_path)
df_main.head()

,CDate,JDate,GroupNo,GroupName,MarketValue,TransactionsCount,TransactionsVol,TransactionsValue
0,2019-06-03,1398-03-13,0,شاخص صنعت,7650169488,236046,2740.000,10212891.0
1,2019-06-03,1398-03-13,34,34-خودرو,269025262,62425,1399.000,2610194.0
2,2019-06-03,1398-03-13,57,57-بانكها,644373421,50274,1447.000,1194232.0
3,2019-06-03,1398-03-13,44,44-شيميايي,1900527339,20390,172.604,989110.0
4,2019-06-03,1398-03-13,27,27-فلزات اساسي,1389537365,15898,112.454,729291.0


In [24]:
# find the group number using regex
def parse_group_no(text):
    if re.search('\d+', text):
        group_no = re.findall('\d+', text)[0]
    else:
        group_no = '0'
        
    return group_no

In [25]:
# prepare a dict of values for creating a DataFrame
CDate = info_c_date * len(values)
CDate = wrap(text=CDate, width=10)
JDate = info_j_date * len(values)
JDate = wrap(text=JDate, width=10)
data = {
    'CDate': CDate,
    'JDate': JDate,
    'GroupNo': [parse_group_no(values[i][0]) for i in range(len(values))],
    'GroupName': [(values[i][0]).encode('utf-8') for i in range(len(values))],
    'MarketValue': [values[i][1] for i in range(len(values))],
    'TransactionsCount': [values[i][2] for i in range(len(values))],
    'TransactionsVol': [values[i][3] for i in range(len(values))],
    'TransactionsValue': [values[i][4] for i in range(len(values))]
}

In [26]:
data

{'CDate': ['2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09',
  '2019-06-09'],
 'JDate': ['1398-03-19',
  '1398-03-19',
  '1398-03-19',
  '1398-03-19',
  '1398-03-19',
  '1398-03-19',
  '1398-03-19',
  '1398-03-19',
  '1398-03-19',
  '1398-03-19',
  '1398-03-19',
  '1398-03-19',
  '1398-03-19',
  '1398-03-19',
  '1398-03-19',
  '1398-03-19',
  '1398-03-19',
  '1398-03-19',
  '1398-03-19',
  '1398-03-19',
  '13

In [27]:
# create new DataFrame with recently fetched data
df = pd.DataFrame(data=data)
df.head()

,CDate,JDate,GroupNo,GroupName,MarketValue,TransactionsCount,TransactionsVol,TransactionsValue
0,2019-06-09,1398-03-19,0,b'\xd8\xb4\xd8\xa7\xd8\xae\xd8\xb5 \xd8\xb5\xd...,7.791564e+09,211661.0,2099,9.25045e+06
1,2019-06-09,1398-03-19,34,b'34-\xd8\xae\xd9\x88\xd8\xaf\xd8\xb1\xd9\x88',2.831947e+08,31946.0,641.540,1.43995e+06
2,2019-06-09,1398-03-19,44,b'44-\xd8\xb4\xd9\x8a\xd9\x85\xd9\x8a\xd8\xa7\...,1.897530e+09,23884.0,208.416,1.09554e+06
3,2019-06-09,1398-03-19,57,b'57-\xd8\xa8\xd8\xa7\xd9\x86\xd9\x83\xd9\x87\...,6.514490e+08,35764.0,1022,1.00772e+06
4,2019-06-09,1398-03-19,43,b'43-\xd9\x85\xd9\x88\xd8\xa7\xd8\xaf \xd8\xaf...,3.402371e+08,16158.0,108.873,859619


In [28]:
# read the original excel file
df_main = pd.read_excel(file_path)
df_main.head()

,CDate,JDate,GroupNo,GroupName,MarketValue,TransactionsCount,TransactionsVol,TransactionsValue
0,2019-06-03,1398-03-13,0,شاخص صنعت,7650169488,236046,2740.000,10212891.0
1,2019-06-03,1398-03-13,34,34-خودرو,269025262,62425,1399.000,2610194.0
2,2019-06-03,1398-03-13,57,57-بانكها,644373421,50274,1447.000,1194232.0
3,2019-06-03,1398-03-13,44,44-شيميايي,1900527339,20390,172.604,989110.0
4,2019-06-03,1398-03-13,27,27-فلزات اساسي,1389537365,15898,112.454,729291.0


In [29]:
# combine recently created DataFrame and the original one
df1 = pd.concat([df_main, df], ignore_index=True, sort=False)
df1.head()

,CDate,JDate,GroupNo,GroupName,MarketValue,TransactionsCount,TransactionsVol,TransactionsValue
0,2019-06-03,1398-03-13,0,شاخص صنعت,7.650169e+09,236046.0,2740,1.02129e+07
1,2019-06-03,1398-03-13,34,34-خودرو,2.690253e+08,62425.0,1399,2.61019e+06
2,2019-06-03,1398-03-13,57,57-بانكها,6.443734e+08,50274.0,1447,1.19423e+06
3,2019-06-03,1398-03-13,44,44-شيميايي,1.900527e+09,20390.0,172.604,989110
4,2019-06-03,1398-03-13,27,27-فلزات اساسي,1.389537e+09,15898.0,112.454,729291


In [30]:
# Save the output to excel file
df1.to_excel(excel_writer=file_path, index=False)